In [34]:
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms import VQE, QAOA
from qiskit.algorithms.optimizers import SPSA, SLSQP
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.applications import Tsp
import networkx as nx

In [26]:
G = nx.Graph()
G.add_edge(0, 1, weight=1)
G.add_edge(0, 2, weight=2)
G.add_edge(0, 3, weight=3)
G.add_edge(1, 2, weight=4)
G.add_edge(1, 3, weight=5)
G.add_edge(2, 3, weight=6)

In [27]:
tsp = Tsp(G)
qp = tsp.to_quadratic_program()
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubit_op, offset = qubo.to_ising()
print("Distances: ", nx.adjacency_matrix(G))

Distances:    (0, 1)	1
  (0, 2)	2
  (0, 3)	3
  (1, 0)	1
  (1, 2)	4
  (1, 3)	5
  (2, 0)	2
  (2, 1)	4
  (2, 3)	6
  (3, 0)	3
  (3, 1)	5
  (3, 2)	6


/var/folders/f4/n3_mnvbd6q50h0gtzy84vyn40000gn/T/ipykernel_17223/3849291369.py:6: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  print("Distances: ", nx.adjacency_matrix(G))


In [ ]:
seed = 10
from mqt import ddsim
backend = ddsim.DDSIMProvider().get_backend('qasm_simulator')
#backend = Aer.get_backend("aer_simulator")

quantum_instance = QuantumInstance(
    backend, seed_simulator=seed, shots=1024
)

ansatz = TwoLocal(qubit_op.num_qubits, "ry", "cz", reps=7, entanglement="linear")
#vqe = VQE(ansatz, optimizer=SPSA(maxiter=100), quantum_instance=quantum_instance)
#vqe_result = vqe.compute_minimum_eigenvalue(qubit_op)
qaoa = QAOA(reps=2, optimizer=SLSQP(maxiter=25), quantum_instance=quantum_instance)
qaoa_result = qaoa.compute_minimum_eigenvalue(qubit_op)




In [ ]:
x = tsp.sample_most_likely(qaoa_result.eigenstate)
print("feasible:", qubo.is_feasible(x))
z = tsp.interpret(x)
print("solution:", z)
print(x)